In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher

In [64]:
## here we give path ,name . 
## name we give folder name  we want in path dir

path = "workdata/lines_submit/wrong_lines/sorteddata/"   #  or "workdata/lines_submit/correct_lines/sorteddata/" 
name = "1086"


In [65]:
fname1 = path+name+"/dataframe.txt"                     ## before moodified so we take fname5 instead of it
fname2 = path+name+"/input_line.txt"
fname3 = path+name+"/original_line.txt"
fname5 = path+name+"/dataframe_withsandhi.txt"
fname6 = path+name+"/conflicts.csv"
fname7 = path+name+"/problem_words.txt"

In [66]:
def printoriginalline(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if line.startswith( 'SENT: '):
                print(line)
                break
            print(line)
    return

printoriginalline(fname3)



filename : RSVP_karNa_GOLD.txt

lineno : 538

SENT: anekAsu rAwriRu mama saMmugXanixrAvasWAyAm sA AgawA 



In [67]:
def printlines(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            print(line)
    return


printlines(fname2)

input line : anekAsu rAwriRu mama saMmugXanixrAvasWAyAm sA AgawA 

converted line : anekāsu rātriṣu mama sammugdhanidrāvasthāyām sā_āgatā 

converted line char :a,n,e,k,ā,s,u, ,r,ā,t,r,i,ṣ,u, ,m,a,m,a, ,s,a,m,m,u,g,dh,a,n,i,d,r,ā,v,a,s,th,ā,y,ā,m, ,s,ā,_,ā,g,a,t,ā, 



In [68]:
def dependency_structure(fname):
    ds = pd.read_csv(fname,encoding = 'utf-8' ,sep = '\t',skiprows = 4,header=None)
    return ds


ds = dependency_structure(fname3)
ds

,0,1,2,3
0,1,anekAsu,NV,"viSeRaNam,2"
1,2,rAwriRu,NP,"aXikaraNam,6"
2,3,mama,SN,"RaRTIsambanXaH,4"
3,4,saMmugXanixrAvasWAyAm,NaN,"aXikaraNam,6"
4,5,sA,SN,"karwA,6"
5,6,AgawA,KP,NaN


In [69]:
def fetched_data(fname) :
    fd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return fd


fd = fetched_data(fname5)
fd

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf,word_slp1,sandhi_indexs,sandhi_words
0,0,1,light_blue_back,0,1,anekāsu,aneka,NaN,loc. pl. f.,7,7,NaN,anekAsu,NaN,NaN
1,1,1,deep_sky_back,8,2,rātriṣu,rātri,NaN,loc. pl. f.,7,7,NaN,rAtrizu,NaN,NaN
2,2,1,light_blue_back,16,3,mama,asmad,NaN,g. sg. *,4,4,NaN,mama,NaN,NaN
3,3,1,yellow_back,21,4,sammugdha,mugdha,sam,iic.,8,9,{ pp. }[sam-muh],sammugDa,NaN,NaN
4,4,1,yellow_back,29,4,nidrā,nidrā,NaN,iic.,5,5,sence of lemma = 2,nidrA,"14, 17, 21, 23, 25, 26, 27,","avasthāyām(avasTAyAm), avasthā(avasTA), avasth..."
5,5,1,carmin_back,36,4,stha,as,NaN,pr. [2] ac. pl. 2,3,4,sence of lemma = 1,sTa,"12, 24, 28,","ayām(ayAm), āyām(AyAm), ayām(ayAm),"
6,6,1,cyan_back,39,4,yām,ya,NaN,acc. sg. f.,3,3,sence of lemma = 2,yAm,NaN,NaN
7,7,1,grey_back,43,5,sā_āgatā,sā,NaN,?,8,8,sence of lemma = āgatā,sA_AgatA,NaN,NaN
8,8,2,yellow_back,0,1,aneka,aneka,NaN,iic.,5,5,NaN,aneka,"19, 20, 22,","asu(asu), asu(asu), āsu(Asu),"
9,9,2,carmin_back,16,3,mama,mā,NaN,pft. ac. pl. 2,4,4,sence of lemma = 1,mama,NaN,NaN


In [70]:
def conflicts_dataframe(fname) :
    cd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return cd


cd = conflicts_data(fname6)
cd
## here 2 == conflict ,1 = sandhi possible  , 0 = not conflict
## same word in same level , same position with different index are not taken as conflict (as it represent same word only)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,2,0,...,2,2,0,2,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,2
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
9,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
## here we give index as input and get conflicted words dataframe
def conflict_for_index(i):
    ci = fd.loc[cd.loc[cd[str(i)] == 2].index ]
    return ci

ci = conflict_for_index(8)
ci
## here we only printed conflicted ie checked value == 2

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf,word_slp1,sandhi_indexs,sandhi_words
0,0,1,light_blue_back,0,1,anekāsu,aneka,NaN,loc. pl. f.,7,7,NaN,anekAsu,NaN,NaN
13,13,3,light_blue_back,0,1,anekā,aneka,NaN,nom. sg. f.,5,5,NaN,anekA,"19, 20, 22,","asu(asu), asu(asu), āsu(Asu),"
15,15,4,yellow_back,0,1,an,an,NaN,iic.,2,2,sence of lemma = 1,an,NaN,NaN
16,16,4,yellow_back,2,1,eka,eka,NaN,iic.,3,3,NaN,eka,"19, 20, 22,","asu(asu), asu(asu), āsu(Asu),"
